In [1]:
import pandas as pd
import requests
from itertools import combinations

In [2]:
api_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    print("Succesfully fetched data!")
else:
    print(f"Error while fetching data. Status code: {response.status_code}")

Succesfully fetched data!


In [3]:
print(data.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [4]:
players = pd.DataFrame(data['elements'])
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,137,82,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,...,32,15,303,38,459,49,210,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,...,66,3,76,17,31,4,14,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,...,13,5,88,18,37,14,36,9,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,638,63,425,51,514,58,558,68,0.00,0.00


In [5]:
print("Players DataFrame Columns:")
print(players.columns.tolist())


Players DataFrame Columns:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_

In [6]:
position_map = {
    1: 'GK',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

price_updates = {
    'Mohamed Salah': 13.0,
    'Cole Palmer': 10.8,
    'Bukayo Saka': 10.4,
    'Nicolas Jackson': 7.9, 
}

players_selected = players[
    ['first_name', 'second_name', 'team_code', 'element_type', 'total_points', 'points_per_game', 'now_cost']
].copy()

players_selected['now_cost'] = players_selected['now_cost'] / 10

players_selected['element_type'] = players_selected['element_type'].map(position_map)

players_selected[['total_points', 'points_per_game', 'now_cost']] = players_selected[
    ['total_points', 'points_per_game', 'now_cost']
].apply(pd.to_numeric, errors='coerce')

players_selected = players_selected.loc[players_selected['points_per_game'] > 0.0]

players_selected['full_name'] = players_selected['first_name'] + ' ' + players_selected['second_name']

# Remove Bryan Mbeumo's row from the DataFrame
#mbeumo_row = players_selected.loc[players_selected['full_name'] == 'Bryan Mbeumo'].copy()
#players_selected = players_selected.loc[players_selected['full_name'] != 'Bryan Mbeumo']

# Update player prices that have been already selected at a different price
for player, new_price in price_updates.items():
    players_selected.loc[players_selected['full_name'] == player, 'now_cost'] = new_price

players_selected['Cost/Points'] = (players_selected['now_cost'] / players_selected['total_points']).round(2)

print(players_selected)



         first_name             second_name  team_code element_type  \
1           Gabriel       Fernando de Jesus          3      Forward   
2           Gabriel    dos Santos Magalhães          3     Defender   
3               Kai                 Havertz          3      Forward   
5           Jurriën                  Timber          3     Defender   
6        Jorge Luiz            Frello Filho          3   Midfielder   
..              ...                     ...        ...          ...   
686          Jørgen           Strand Larsen         39      Forward   
687    Toti António                   Gomes         39     Defender   
689           André  Trindade da Costa Neto         39   Midfielder   
690  Carlos Roberto            Forbs Borges         39   Midfielder   
691           Alfie                    Pond         39     Defender   

     total_points  points_per_game  now_cost                     full_name  \
1               9              0.8       6.8     Gabriel Fernando de 

In [7]:
max_indeces = players_selected[['total_points', 'points_per_game', 'now_cost']].idxmax()
max_rows = players_selected.loc[max_indeces]
print(max_rows)


    first_name second_name  team_code element_type  total_points  \
401    Mohamed       Salah         14   Midfielder           151   
401    Mohamed       Salah         14   Midfielder           151   
428     Erling     Haaland         43      Forward            98   

     points_per_game  now_cost       full_name  Cost/Points  
401             10.8      13.0   Mohamed Salah         0.09  
401             10.8      13.0   Mohamed Salah         0.09  
428              6.5      14.9  Erling Haaland         0.15  


In [8]:
# Create a sorted copy of the DataFrame
sorted_players = players_selected.sort_values(by='now_cost', ascending=True)
sorted_players = sorted_players.reset_index(drop=True)

# Display the sorted DataFrame
sorted_players.head()


,first_name,second_name,team_code,element_type,total_points,points_per_game,now_cost,full_name,Cost/Points
0,Alfie,Pond,39,Defender,1,1.0,3.9,Alfie Pond,3.90
1,Yerson,Mosquera,39,Defender,3,0.6,3.9,Yerson Mosquera,1.30
2,James,Hill,91,Defender,1,1.0,3.9,James Hill,3.90
3,Charlie,Taylor,20,Defender,6,1.0,3.9,Charlie Taylor,0.65
4,Ronnie,Edwards,20,Defender,1,1.0,3.9,Ronnie Edwards,3.90


In [14]:
# Pick seed optimal team
goalkeepers = sorted_players[sorted_players['element_type'] == 'GK']
defenders = sorted_players[sorted_players['element_type'] == 'Defender']
midfielders = sorted_players[sorted_players['element_type'] == 'Midfielder']
forwards = sorted_players[sorted_players['element_type'] == 'Forward']

g = 2
d = 5
m = 5
s = 3

seed_starters = list(goalkeepers.head(g).index) + list(defenders.head(d).index) + list(midfielders.head(m).index) + list(forwards.head(s).index)

print(seed_starters)

best_total_points = sorted_players.loc[seed_starters, 'total_points'].sum()
best_comb_cost = sorted_players.loc[seed_starters, 'now_cost'].sum()
best_cpp = best_comb_cost / best_total_points

# Print the team
print(f"Total points: {best_total_points}")
print(f"Total cost: £{best_comb_cost}m")
print(f"Cost/point: {best_cpp:.2f}m\n")

for player_idx in seed_starters:
    player = sorted_players.loc[player_idx]
    print(f"Player: {player['first_name']} {player['second_name']} - {player['element_type']} - Team Code: {player['team_code']} - "
          f"Total points: {player['total_points']} - Cost: £{player['now_cost']}m - Cost / points: {player['now_cost'] / player['total_points']:.4f}m")

midfielders.head()


[19, 21, 0, 1, 2, 3, 4, 56, 77, 78, 81, 83, 111, 144, 158]
Total points: 45
Total cost: £62.8m
Cost/point: 1.40m

Player: Vítezslav Jaros - GK - Team Code: 14 - Total points: 1 - Cost: £4.0m - Cost / points: 4.0000m
Player: Joe Lumley - GK - Team Code: 20 - Total points: 8 - Cost: £4.0m - Cost / points: 0.5000m
Player: Alfie Pond - Defender - Team Code: 39 - Total points: 1 - Cost: £3.9m - Cost / points: 3.9000m
Player: Yerson Mosquera - Defender - Team Code: 39 - Total points: 3 - Cost: £3.9m - Cost / points: 1.3000m
Player: James Hill - Defender - Team Code: 91 - Total points: 1 - Cost: £3.9m - Cost / points: 3.9000m
Player: Charlie Taylor - Defender - Team Code: 20 - Total points: 6 - Cost: £3.9m - Cost / points: 0.6500m
Player: Ronnie Edwards - Defender - Team Code: 20 - Total points: 1 - Cost: £3.9m - Cost / points: 3.9000m
Player: Hamza Choudhury - Midfielder - Team Code: 13 - Total points: 1 - Cost: £4.3m - Cost / points: 4.3000m
Player: Eric da Silva Moreira - Midfielder - Team

,first_name,second_name,team_code,element_type,total_points,points_per_game,now_cost,full_name,Cost/Points
56,Hamza,Choudhury,13,Midfielder,1,1.0,4.3,Hamza Choudhury,4.30
77,Eric,da Silva Moreira,17,Midfielder,2,1.0,4.4,Eric da Silva Moreira,2.20
78,Ibrahim,Sangaré,17,Midfielder,4,1.3,4.4,Ibrahim Sangaré,1.10
81,Jakub,Moder,36,Midfielder,2,1.0,4.4,Jakub Moder,2.20
83,Boubakary,Soumaré,13,Midfielder,9,1.0,4.4,Boubakary Soumaré,0.49


In [15]:
# Knapsack algorithm

def find_optimal_team(max_cost, seed_team, all_players, starters=[]):
    optimal_team = seed_team.copy()
    sorted_players = all_players.sort_values(by='Cost/Points', ascending=True)
    for idx, row in sorted_players.iterrows():
        if idx not in optimal_team + starters:
            # Find players in same position
            same_pos_players = all_players.loc[optimal_team]
            same_pos_players = same_pos_players[same_pos_players['element_type'] == row['element_type']]

            if same_pos_players.empty:
                continue

            min_points_player = same_pos_players['total_points'].idxmin()

            if row['total_points'] > all_players.loc[min_points_player, 'total_points']:
                worst_player_price = all_players.loc[min_points_player, 'now_cost']
                curr_cost = all_players.loc[optimal_team, 'now_cost'].sum() - worst_player_price
                if curr_cost + row['now_cost'] <= max_cost:
                    # Replace the optimal player
                    optimal_team.remove(min_points_player)
                    optimal_team.append(idx)

    return optimal_team


In [17]:
max_cost = 100
all_players = sorted_players.copy()
starting_team = find_optimal_team(max_cost, seed_starters, all_players)

print(starting_team)

# Calculate team statistics
best_total_points = sorted_players.loc[starting_team, 'total_points'].sum()
best_comb_cost = sorted_players.loc[starting_team, 'now_cost'].sum()
best_cpp = best_comb_cost / best_total_points

# Print the team
print(f"Total points: {best_total_points}")
print(f"Total cost: £{best_comb_cost}m")
print(f"Cost/point: {best_cpp:.2f}m")

# Define the order of positions
position_order = ['GK', 'Defender', 'Midfielder', 'Forward']

# Iterate through each position and print players
for position in position_order:
    print(f"\nPosition: {position}")
    for player_idx in starting_team:
        player = sorted_players.loc[player_idx]
        if player['element_type'] == position:
            print(f"Player: {player['first_name']} {player['second_name']} - {player['element_type']} - Team Code: {player['team_code']} - "
                f"Total points: {player['total_points']} - Cost: £{player['now_cost']}m - Cost / points: {player['now_cost'] / player['total_points']:.4f}m")

[432, 181, 240, 166, 387, 459, 325, 167, 142, 450, 188, 471, 472, 462, 460]
Total points: 1167
Total cost: £99.4m
Cost/point: 0.09m

Position: GK
Player: Matz Sels - GK - Team Code: 17 - Total points: 62 - Cost: £4.7m - Cost / points: 0.0758m
Player: André Onana - GK - Team Code: 1 - Total points: 65 - Cost: £5.2m - Cost / points: 0.0800m

Position: Defender
Player: Ola Aina - Defender - Team Code: 17 - Total points: 58 - Cost: £4.9m - Cost / points: 0.0845m
Player: Ashley Young - Defender - Team Code: 11 - Total points: 57 - Cost: £4.6m - Cost / points: 0.0807m
Player: Murillo Santiago Costa dos Santos - Defender - Team Code: 17 - Total points: 53 - Cost: £4.6m - Cost / points: 0.0868m
Player: Nikola Milenković - Defender - Team Code: 17 - Total points: 50 - Cost: £4.5m - Cost / points: 0.0900m
Player: Daniel Muñoz - Defender - Team Code: 31 - Total points: 50 - Cost: £4.7m - Cost / points: 0.0940m

Position: Midfielder
Player: Alex Iwobi - Midfielder - Team Code: 54 - Total points: 7

In [12]:
max_cost2 = 100 - max_cost
all_players = sorted_players.copy()

seed_bench = list(goalkeepers.head(1).index) + list(defenders.head(5-d).index) + list(midfielders.head(5-m).index) + list(forwards.head(3-s).index)
bench = find_optimal_team(max_cost2, seed_bench, all_players, starting_team)

# Calculate team statistics
best_total_points = sorted_players.loc[bench, 'total_points'].sum()
best_comb_cost = sorted_players.loc[bench, 'now_cost'].sum()
best_cpp = best_comb_cost / best_total_points

# Print the team
print(f"Total points: {best_total_points}")
print(f"Total cost: £{best_comb_cost}m")
print(f"Cost/point: {best_cpp:.2f}m")

# Define the order of positions
position_order = ['GK', 'Defender', 'Midfielder', 'Forward']

# Iterate through each position and print players
for position in position_order:
    print(f"\nPosition: {position}")
    for player_idx in bench:
        player = sorted_players.loc[player_idx]
        if player['element_type'] == position:
            print(f"Player: {player['first_name']} {player['second_name']} - {player['element_type']} - Team Code: {player['team_code']} - "
                f"Total points: {player['total_points']} - Cost: £{player['now_cost']}m - Cost / points: {player['now_cost'] / player['total_points']:.4f}m")

Total points: 92
Total cost: £17.0m
Cost/point: 0.18m

Position: GK
Player: Matz Sels - GK - Team Code: 17 - Total points: 62 - Cost: £4.7m - Cost / points: 0.0758m

Position: Defender
Player: Dara O'Shea - Defender - Team Code: 40 - Total points: 19 - Cost: £4.0m - Cost / points: 0.2105m
Player: Ben Johnson - Defender - Team Code: 40 - Total points: 8 - Cost: £3.9m - Cost / points: 0.4875m

Position: Midfielder

Position: Forward
Player: Ross Stewart - Forward - Team Code: 20 - Total points: 3 - Cost: £4.4m - Cost / points: 1.4667m


In [13]:
best_points = 0
final_team = None
selected_bench = None
selected_starters = None
seed_bench = list(goalkeepers.head(1).index) + list(defenders.head(5-d).index) + list(midfielders.head(5-m).index) + list(forwards.head(3-s).index)

for i in range(80, 100):
    starters_budget = i
    all_players = sorted_players.copy()
    starting_team = find_optimal_team(max_cost, seed_starters, all_players)
    bench = find_optimal_team(max_cost2, seed_bench, all_players, starting_team)

    starters_total_points = sorted_players.loc[starting_team, 'total_points'].sum()
    bench_total_points = sorted_players.loc[bench, 'total_points'].sum()

    combined_points = starters_total_points + bench_total_points * 0.4

    if combined_points > best_points:
        selected_bench = bench
        selected_starters = starting_team
        final_team = starting_team + bench


# Calculate team statistics
best_total_points = sorted_players.loc[final_team, 'total_points'].sum()
best_comb_cost = sorted_players.loc[final_team, 'now_cost'].sum()
best_cpp = best_comb_cost / best_total_points

# Print the team
print(f"Total points: {best_total_points}")
print(f"Total cost: £{best_comb_cost}m")
print(f"Cost/point: {best_cpp:.2f}m\n")

cost_st = sorted_players.loc[selected_starters, 'now_cost'].sum()
cost_be = sorted_players.loc[selected_bench, 'now_cost'].sum()

print(f"Starters cost: £{cost_st}m")
print(f"Bench cost: £{cost_be}m")

# Define the order of positions
position_order = ['GK', 'Defender', 'Midfielder', 'Forward']

# Iterate through each position and print players
for position in position_order:
    print(f"\nPosition: {position}")
    for player_idx in final_team:
        player = sorted_players.loc[player_idx]
        if player['element_type'] == position:
            print(f"Player: {player['first_name']} {player['second_name']} - {player['element_type']} - Team Code: {player['team_code']} - "
                f"Total points: {player['total_points']} - Cost: £{player['now_cost']}m - Cost / points: {player['now_cost'] / player['total_points']:.4f}m")

Total points: 1044
Total cost: £99.30000000000001m
Cost/point: 0.10m

Starters cost: £82.3m
Bench cost: £17.0m

Position: GK
Player: André Onana - GK - Team Code: 1 - Total points: 65 - Cost: £5.2m - Cost / points: 0.0800m
Player: Matz Sels - GK - Team Code: 17 - Total points: 62 - Cost: £4.7m - Cost / points: 0.0758m

Position: Defender
Player: Ola Aina - Defender - Team Code: 17 - Total points: 58 - Cost: £4.9m - Cost / points: 0.0845m
Player: Ashley Young - Defender - Team Code: 11 - Total points: 57 - Cost: £4.6m - Cost / points: 0.0807m
Player: Murillo Santiago Costa dos Santos - Defender - Team Code: 17 - Total points: 53 - Cost: £4.6m - Cost / points: 0.0868m
Player: Dara O'Shea - Defender - Team Code: 40 - Total points: 19 - Cost: £4.0m - Cost / points: 0.2105m
Player: Ben Johnson - Defender - Team Code: 40 - Total points: 8 - Cost: £3.9m - Cost / points: 0.4875m

Position: Midfielder
Player: Bryan Mbeumo - Midfielder - Team Code: 94 - Total points: 96 - Cost: £7.6m - Cost / po